<a href="https://colab.research.google.com/github/SiluniKeerthiratne/DSGP/blob/nisal/%20v3%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imports
import tensorflow as tf
import os
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras import regularizers
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
# Constants
ROOT_DIR = r'C:\Users\nisal\Notebooks/fruit-dta'
FRUIT_CLASSES = ['banana', 'mango', 'tomato']
RIPENESS_CLASSES = ['0 unripe', '1 partially ripe', '2 ripe']
IMG_ROWS, IMG_COLS = 244, 244
NUM_CLASSES = len(RIPENESS_CLASSES)

In [ ]:
# Function to load and preprocess images
def load_images():
    X = []
    y = []
    for fruit_class in FRUIT_CLASSES:
        for ripeness_class_label, ripeness_class in enumerate(RIPENESS_CLASSES):
            class_path = os.path.join(ROOT_DIR, fruit_class, ripeness_class)
            for img_name in tqdm(os.listdir(class_path), desc=f"Loading {fruit_class}/{ripeness_class}"):
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (IMG_ROWS, IMG_COLS))
                X.append(img)
                y.append(ripeness_class_label)
    return X, y

In [ ]:
# Load and preprocess images
X, y = load_images()

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Print data shapes
print("X_train Shape:", X_train.shape)
print("y_train Shape:", y_train.shape)
print("X_test Shape:", X_test.shape)
print("y_test Shape:", y_test.shape)

In [ ]:
# Load InceptionV3 model with pre-trained weights
inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_ROWS, IMG_COLS, 3))

In [ ]:
# Freeze the layers
for layer in inception.layers:
    layer.trainable = False

# Custom head for the model
def custom_head(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(top_model)
    top_model = Dropout(0.5)(top_model)
    top_model = Dense(num_classes, activation='softmax')(top_model)
    return top_model

# Add custom head on top of InceptionV3
FC_Head = custom_head(inception, NUM_CLASSES)
model = Model(inputs=inception.input, outputs=FC_Head)

In [ ]:
# Model summary
print(model.summary())

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=25, verbose=1, initial_epoch=0, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy * 100, "%")

In [ ]:
# Generate classification report
true_labels = y_test
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=-1)
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=RIPENESS_CLASSES))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
# Function to plot confusion matrix
def plot_confusion_matrix(true_labels, predicted_labels, classes):
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Plot confusion matrix
plot_confusion_matrix(true_labels, predicted_labels, classes=RIPENESS_CLASSES)

In [ ]:
# Plot test accuracy and loss graphs
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Define the path to save the model
model_path = r"C:\Users\nisal\Notebooks\v3_model_9cls.h5"

# Save the trained model
model.save(model_path)
print("Model saved successfully at:", model_path)